### load derived data (json file) into pandas

- load data downloaded from WOS

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import jsonlines

In [2]:
dataPath = "../data/derived1/"
dataFile = "records.jsonl"

In [15]:
test = pd.read_json(path_or_buf=dataPath+dataFile,
                   #orient="table",
                   lines=True,
                   #orient="columns",
                   typ="series"
                   )
#data = pd.DataFrame()
index = 0
with jsonlines.open(dataPath+dataFile) as reader:
    
    for obj in reader:
        if index==0:
            #print(obj["record"])
            test = pd.Series(obj["record"]["static_data"]["summary"])
            index=1
        




In [65]:
#'find if this is an article
#obj["record"]["static_data"]["summary"]["doctypes"]["doctype"]

#find publication title
#obj["record"]["static_data"]["summary"]["titles"]["title"][-1]["content"]

#find journal title
#obj["record"]["static_data"]["summary"]["titles"]["title"][0]["content"]

#find publisher (elsevier, nature, etc)
#obj["record"]["static_data"]["summary"]["publishers"]["publisher"]["names"]["name"]["unified_name"]



'Elsevier'

### plot for the paper:
- number of papers in OSH over years
  - divide them per area (using research areas tags)
    - Engineering, life sciences, humanities?
- quality of the papers - how many fit the OSHWA OSH classification?
- Accessibility of the papers - how many are OA?